In [1]:
#https://www.researchgate.net/publication/281521039_An_Integrated_Framework_Advancing_Membrane_Protein_Modeling_and_Design/figures?lo=1
from pyrosetta.rosetta import *
from pyrosetta.toolbox import *
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from matplotlib import pyplot as plt
import numpy as np

In [2]:
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.python37.Release r242 2020.02+release.22ef835b4a2 22ef835b4a2647af94fcd6421a85720f07eddf12 http://www.pyrosetta.org 2020-01-05T17:31:56
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /Users/tianhao/anaconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=1093657036 seed_offset=0 real_seed=1093657036 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=1093657036 RG_type=mt19937


In [3]:
ori_pose = pose_from_pdb('3B7E_filtered.pdb')

core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.  Created 980 residue types
core.chemical.GlobalResidueTypeSet: {0} Total time to initialize 1.6536 seconds.
core.import_pose.import_pose: {0} File '3B7E_filtered.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 10 335
core.conformation.Conformation: {0} current variant for 10 CYS
core.conformation.Conformation: {0} current variant for 335 CYS
core.conformation.Conformation: {0} current variant for 10 CYD
core.conformation.Conformation: {0} current variant for 335 CYD
core.conformation.Conformation: {0} Found disulfide between residues 42 47
core.conformation.Conformation: {0} current variant for 42 CYS
core.conformation.Conformation: {0} current variant for 47 CYS
core.conformation.Conformation: {0} current variant for 42 CYD
core.conformation.Conformation: {0} current variant for 47 CYD
core.conformation.Conformation: {0} Found

In [4]:
#score functions could be optimized
sfxn = create_score_function("ddg_monomer")
ori_pose.sequence()

core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.

'VILTGNSSLCPISGWAIYSKDNGIRIGSKGDVFVIREPFISCSHLECRTFFLTQGALLNDKHSNGTVKDRSPYRTLMSCPVGEAPSPYNSRFESVAWSASACHDGMGWLTIGISGPDNGAVAVLKYNGIITDTIKSWRNNILRTQESECACVNGSCFTIMTDGPSNGQASYKILKIEKGKVTKSIELNAPNYHYEECSCYPDTGKVMCVCRDNWHGSNRPWVSFDQNLDYQIGYICSGVFGDNPRPNDGTGSCGPVSSNGANGIKGFSFRYDNGVWIGRTKSTSSRSGFEMIWDPNGWTETDSSFSVRQDIVAITDWSGYSGSFVQHPELTGLDCMRPCFWVELIRGQPKENTIWTSGSSISFCGVNSDTVGWSWPDGAELPFSI'

In [5]:
#movers could be optimzied
min_mover = rosetta.protocols.minimization_packing.MinMover()
#min_mover = rosetta.protocols.minimization_packing.PackRotamersMover()
mm = MoveMap()
mm.set_bb(True)
mm.set_chi(True)
min_mover.movemap(mm)
min_mover.score_function(sfxn)
min_mover.min_type('linmin')

In [6]:
def compute_ddG( pose, sfxn, muts, repack_radius):

    #Score Native Pose
    n_pose = Pose()
    n_pose.assign(pose)
    aa = muts[0]
    resnum = int(muts[1:-1])-82
    mutate_residue( n_pose, resnum, aa, repack_radius, sfxn ) 
    min_mover.apply(n_pose)
    native_score = sfxn(n_pose)
    

    resnum = int(muts[1:-1])-82
    aa = muts[-1]
    mutate_residue( pose, resnum, aa, repack_radius, sfxn ) 
       
    
    # Score Mutated Pose
    #min_mover do not converge
    min_mover.apply(pose)
    mutant_score = sfxn( pose )

    # return scores
    return muts, round( mutant_score, 3 ), round( native_score, 3 ), mutant_score - native_score

In [7]:
infile = open('MutList_3b7e_NA')
mutlist = []
for line in infile:
    line = line.rstrip()
    mutlist.append(line)
infile.close()

In [7]:
mutlist

['V83A',
 'V83C',
 'V83D',
 'V83E',
 'V83F',
 'V83G',
 'V83H',
 'V83I',
 'V83K',
 'V83L',
 'V83M',
 'V83N',
 'V83P',
 'V83Q',
 'V83R',
 'V83S',
 'V83T',
 'V83W',
 'V83Y',
 'V83X',
 'V83Z',
 'I84A',
 'I84C',
 'I84D',
 'I84E',
 'I84F',
 'I84G',
 'I84H',
 'I84K',
 'I84L',
 'I84M',
 'I84N',
 'I84P',
 'I84Q',
 'I84R',
 'I84S',
 'I84T',
 'I84V',
 'I84W',
 'I84Y',
 'I84X',
 'I84Z',
 'L85A',
 'L85C',
 'L85D',
 'L85E',
 'L85F',
 'L85G',
 'L85H',
 'L85I',
 'L85K',
 'L85M',
 'L85N',
 'L85P',
 'L85Q',
 'L85R',
 'L85S',
 'L85T',
 'L85V',
 'L85W',
 'L85Y',
 'L85X',
 'L85Z',
 'T86A',
 'T86C',
 'T86D',
 'T86E',
 'T86F',
 'T86G',
 'T86H',
 'T86I',
 'T86K',
 'T86L',
 'T86M',
 'T86N',
 'T86P',
 'T86Q',
 'T86R',
 'T86S',
 'T86V',
 'T86W',
 'T86Y',
 'T86X',
 'T86Z',
 'G87A',
 'G87C',
 'G87D',
 'G87E',
 'G87F',
 'G87H',
 'G87I',
 'G87K',
 'G87L',
 'G87M',
 'G87N',
 'G87P',
 'G87Q',
 'G87R',
 'G87S',
 'G87T',
 'G87V',
 'G87W',
 'G87Y',
 'G87X',
 'G87Z',
 'N88A',
 'N88C',
 'N88D',
 'N88E',
 'N88F',
 'N88G',
 

In [8]:
#test if mover converge the score function
#thought not converging, maybe don't run


for i in range(400):
    native_score = sfxn(ori_pose)
    #print(native_score)
    min_mover.apply(ori_pose)
    native_score = sfxn(ori_pose)
    if i % 100 == 0:
        print(native_score)

basic.io.database: {0} Database file opened: scoring/score_functions/disulfides/fa_SS_distance_score
core.pack.dunbrack.RotamerLibrary: {0} shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: {0} shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: {0} Binary rotamer library selected: /Users/tianhao/anaconda3/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: {0} Using Dunbrack library binary file '/Users/tianhao/anaconda3/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: {0} Dunbrack 2010 library took 0.37836 seconds to load from binary
644.3203605663009
628.311399482154
619.8630249717978
619.1944187290342


In [1]:
ddgdict = {}; mutcount = 0; pose = Pose()
for muts in mutlist:
    mutcount += 1
    #if mutcount == 2: break
    print(mutcount)
    ddglist = []
    for i in range(10):
        pose.assign(ori_pose)
        mut_ddg = compute_ddG(pose, sfxn, muts, 8.0)
        ddglist.append(mut_ddg[3])
    ddgdict[muts] = ddglist
    #ddgdict[muts] = min(ddglist)

NameError: name 'Pose' is not defined

In [10]:
outfile = open('ddg_3B7E.txt','w')
for mut in ddgdict:
    ddglist = ddgdict[mut]
    outfile.write(mut+'\t'+'\t'.join([str(x) for x in ddglist])+'\n')
outfile.close()
outfile = open('ddg_3B7E_mean.txt','w')
for mut in ddgdict:
    ddglist = ddgdict[mut]
    outfile.write(mut+'\t'+str(np.mean(ddglist))+'\n')
outfile.close()

In [ ]:
len(mutlist)

In [39]:
print(ori_pose.sequence())
print(pose.sequence())

GLPFISCQKGYKGVWAGTGIMTTRCPCGANISGNVRLGSMRITGPKTCMNTWQGTFPINCYTEGQCAP
GLPFISCQKGYKGVWCGTGIMTTRCPCGANISGNVRLGSMRITGPKTCMNTWQGTFPINCYTEGQCAP


In [20]:
scorefxn = get_fa_scorefxn()
print('ref2015',scorefxn(pose))
scorefxn = create_score_function("ddg")
print('ddg',scorefxn(pose))
scorefxn = create_score_function("ddg_monomer")
print('ddg_monomer',scorefxn(pose))

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
basic.io.database: {0} Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: {0} Database file opened: scoring/score_functions/rama/fd/prepro.ramaProb
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.all.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.gly.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.pro.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.valile.txt
basic.io.database: {0} Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: {0} Read 40 countpair representative atoms
ref2015 83.72842627781196
core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline